In [57]:
import nltk
import string
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import snowball
from nltk.corpus import stopwords
import PyPDF2
from glob import glob, glob1
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\doonl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\doonl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
path = './txtFile/'

data = pd.DataFrame(columns=['id', 'abstract', 'keywords'])
for i in glob1('./txtFile/', '*'):
    with open(path + i, 'r', encoding='utf8') as f:
        doc_id = i.split('.')[-2]
        readingAbstract = False
        readingKeywords = False
        abstract_sent = []
        keyword_list = []
        keyword_clean = []
        for j in f:
            line = j.strip()
            if line == 'Keywords:' and readingKeywords is False:
                readingKeywords = True
                readingAbstract = False
                continue
            if line == 'Abstract' and readingAbstract is False:
                readingAbstract = True
                continue
            if readingAbstract is True:
                abstract_sent.append(line)
            if readingKeywords is True:
                keyword_list.append(line)
        abstract_para = ' '.join(abstract_sent)
        keyword_split = keyword_list[0].split(',')
        # print(abstract_para)
        # print(keyword_split)
        for x in keyword_split:
            keyword_lower = x.lower()
            keyword_no_space = keyword_lower.replace(' ', '')
            keyword_underscore = keyword_no_space.replace('_', ' ')
            keyword_clean.append(keyword_underscore)
        # print(keyword_clean)

        data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
    f.close()

display(data)


C:\Users\doonl\AppData\Local\Temp\ipykernel_15920\2526147678.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
C:\Users\doonl\AppData\Local\Temp\ipykernel_15920\2526147678.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
C:\Users\doonl\AppData\Local\Temp\ipykernel_15920\2526147678.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"id": doc_id, "abstract": abstract_para, "keywords": keyword_clean}, ignore_index=True)
C:\Users\doonl\AppData\Local\Temp\ipykernel_15920\2526147678.py:36:

,id,abstract,keywords
0,1,"In the logistic management, the cost reduction...","[artificial intelligence, combinatorial optimi..."
1,10,Word segmentation is an important pre-processi...,"[attention, deep learning, thai language, word..."
2,11,Loss estimation is considerably significant fo...,"[network resource planning, traffic model, que..."
3,12,This thesis presents a computational study on...,"[audio-synchronization, facial expression, fac..."
4,13,The financial problem is a challenging task fo...,"[deep learning, distributional reinforcement l..."
5,14,Since a classical computer has a limitation to...,"[circuit depth, computing wall-time, quantum r..."
6,15,Hyperparameter Tuning chooses optimal hyperpar...,"[abc, anova, classification, grid search, hype..."
7,16,The honeybee is a social insect that communica...,"[convolutional neural networks, feature extrac..."
8,17,System logs contain the complete information o...,"[alice, anomaly detection, cern, convolutional..."
9,18,Handling an imbalanced class problem is a chal...,"[classification, imbalanced dataset, machine l..."


In [59]:
data.count()

id          18
abstract    18
keywords    18
dtype: int64

In [60]:
data_explode = data.explode('keywords').reset_index(drop=True)

In [61]:
data_explode

,id,abstract,keywords
0,1,"In the logistic management, the cost reduction...",artificial intelligence
1,1,"In the logistic management, the cost reduction...",combinatorial optimization
2,1,"In the logistic management, the cost reduction...",vehicle routing problem
3,1,"In the logistic management, the cost reduction...",hard time windows
4,1,"In the logistic management, the cost reduction...",meta heuristic
...,...,...,...
78,9,Understanding and interpretation of legislativ...,coreference resolution
79,9,Understanding and interpretation of legislativ...,legal processing
80,9,Understanding and interpretation of legislativ...,natural language processing
81,9,Understanding and interpretation of legislativ...,semantic meaning representations


In [62]:
data_explode[data_explode['keywords'] == 'deep learning'].tail()


,id,abstract,keywords
7,10,Word segmentation is an important pre-processi...,deep learning
19,13,The financial problem is a challenging task fo...,deep learning
57,4,Due to the emerging of Long Short-Term Memory ...,deep learning
66,6,This thesis proposed a speech production acqui...,deep learning


In [68]:
data_explode.describe()

,id,abstract,keywords
count,83,83,83
unique,18,18,70
top,9,Understanding and interpretation of legislativ...,deep learning
freq,7,7,4


In [63]:
x = data_explode['abstract'].values
y = data_explode['keywords'].values

In [64]:
x[0]

'In the logistic management, the cost reduction for delivering the  goods to customer s is crucial to save the budget of the company . Since decades ago, various Vehicle Routing Problems (VRPs ) have been emerged enormously to improve the productivity and to reduce the logistic cost of the industry . Among them, Vehicle Routing Problem with Time Windows  (VRPTW ) is one of the most fundamental VRP variants and one of the most applicable variants in the real-world case studies. In this study , VRPTW with  hard time windows is solved  by developing a special Genetic Algorithm (GA), composed of a problem -specific crossover operator and seven different mutation operators. The proposed GA has better results with  the heuristic mutation among seven operators while exploring the new and better features in large search space. The results of the algorithm are tested on the popular Solomon benchmark 100 - customer datasets. The results show that the proposed GA is quite comparable  with the bes

In [65]:
y[0]

'artificial intelligence'

In [66]:
from keras import *
from keras.preprocessing import text
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [67]:
# Split train & test sets
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.1,stratify=y,random_state=0)
# Split train validate sets
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size =0.1,stratify=y,random_state=0)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.